In [ ]:

import pandas as pd

dtype = {
    'ACCOUNT_CDE': 'str',  
    'ASSETID':'str',
    'BUSINESS_UNIT':'str',
    'BUSINESS_UNIT_CDE': 'str', 
    'BUYER_ID': 'str', 
    'COMPANY_CDE': 'str',
    'COST_CENTER': 'str', 
    'COST_CENTER_DESC': 'str',
    'EXPENSE_REPORT_NAME': 'str',
    'FILE_NAME': 'str',
    'FISCAL_MONTH': 'str',
    'FISCAL_QUARTER': 'str',
    'FISCAL_YEAR': 'str',
    'INVOICE_COMMENT': 'str',
    'INVOICE_PMT_TERMS': 'str',
    'INVOICE_LINE_DESC': 'str',
    'INVOICE_LINE_NBR': 'str', 
    'INVOICE_LINE_QTY': 'str', 
    'INVOICE_LINE_UNIT_PRICE': 'str', 
    'INVOICE_NBR': 'str', 
    'INVOICE_PMT_TERMS_CDE': 'str', 
    'LEGAL_ENTITY_CDE': 'str',
    'LOCATION': 'str', 
    'LOCATION_ID': 'str',
    'PAYMENT_TYPE': 'str',
    'PO_LINE_DESC': 'str',
    'PO_LINE_ITEM_ID': 'str', 
    'PO_LINE_NBR': 'str', 
    'PO_LINE_QTY': 'str', 
    'PO_NBR': 'str', 
    'PROJECT_NAME':'str',
    'PROJECT_DESCRIPTION': 'str',
    'SUPPLIER_ADDRESS_1': 'str',
    'SUPPLIER_GRP': 'str',
    'SUPPLIER_ERP': 'str',
    'SUPPLIER_NBR': 'str',
    'SUPPLIER_NORMALIZED': 'str',
    'SUPPLIER_PMT_TERM_CDE': 'str',
    'SUPPLIER_STATE': 'str',
    'SUPPLIER_TAXID': 'str',
    'SUPPLIER_ZIP_POSTAL_CDE':'str',
    'DAYS_ATF_PO': 'str', 
    'INVOICE_LINE_AMOUNT': 'float',
    'EXCHANGE_RATE': 'float',
    'TOTAL_SPEND_USD': 'float',
    'TOTAL_SPEND':'float',
}

df1 = pd.read_csv('ATF2_TBC_OUTPUT_CONCUR_FY23 Q3_OCT v.2.csv', encoding='UTF-8-SIG', encoding_errors='ignore', dtype=dtype)
df2 = pd.read_csv('ATF3_TBC_OUTPUT_CONCUR_FY23 Q3_OCT v.2.csv', encoding='UTF-8-SIG', encoding_errors='ignore', dtype=dtype)

df1_shape = df1.shape
print("DataFrame1 shape:", df1_shape)

df2_shape = df2.shape
print("DataFrame2 shape:", df2_shape)

# Count of non-null values in 'SUPPLIER_NORMALIZED' in df1
nonnull_count_df1 = df1['SUPPLIER_NORMALIZED'].notna().sum()

# Count of non-null values in 'SUPPLIER_NORMALIZED' in df2
nonnull_count_df2 = df2['SUPPLIER_NORMALIZED'].notna().sum()

print(f"Non-null values in  df1: {nonnull_count_df1}")
print(f"Non-null values in df2: {nonnull_count_df2}")


D:\Users\diana.fernandez\AppData\Local\Temp\1\ipykernel_4160\1435315965.py:53: DtypeWarning: Columns (7,32,57,59,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('TBC_OUTPUT_SAP_FY23_DEC-JAN v.2.csv', encoding='UTF-8-SIG', encoding_errors='ignore', dtype=dtype)


DataFrame1 shape: (8386, 90)
DataFrame2 shape: (64469, 90)
Non-null values in  df1: 8220
Non-null values in df2: 0


In [ ]:
# Define the columns on which you want to merge
merge_keys = ['ASSETID', 'FILE_NAME']

# Perform the full outer join on the specified keys
merged_df = pd.merge(df1, df2, on=merge_keys, how='outer', suffixes=('', '_df2'))

# Now, we need to address the 'SUPPLIER_NORMALIZED' column specifically
# We fill NaNs in 'SUPPLIER_NORMALIZED' from df1 with values from df2
merged_df['SUPPLIER_NORMALIZED'] = merged_df['SUPPLIER_NORMALIZED'].combine_first(merged_df['SUPPLIER_NORMALIZED_df2'])

# For all other columns, we want to keep the original values from df1
# So we drop the duplicate columns from df2
columns_to_drop = [col for col in merged_df if col.endswith('_df2') and col != 'SUPPLIER_NORMALIZED_df2']
merged_df.drop(columns=columns_to_drop, inplace=True)

# We also drop the suffixed 'SUPPLIER_NORMALIZED' column from df2
merged_df.drop(columns=['SUPPLIER_NORMALIZED_df2'], inplace=True)

# Verify the shape of the DataFrame
assert merged_df.shape == (4981, 90), f"Unexpected DataFrame shape: {merged_df.shape}"

# Verify the number of non-null values in 'SUPPLIER_NORMALIZED'
nonnull_count = merged_df['SUPPLIER_NORMALIZED'].notna().sum()
print(f"Shape of the merged DataFrame: {merged_df.shape}")
print(f"Non-null values in 'SUPPLIER_NORMALIZED': {nonnull_count}")



Shape of the merged DataFrame: (4981, 90)
Non-null values in 'SUPPLIER_NORMALIZED': 1533


In [ ]:
merged_df.to_csv('ATF3_TBC_OUTPUT_CONCUR_FY23 Q3_OCT v.2.csv', encoding='UTF-8-SIG', index=False)